In [1]:
# Load all python libraries upfront
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import pickle
import sys

In [2]:
# Sentiment analysis on the IMDB movie dataset (https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
# Below is the path to the downloaded labelled file
data_path = '/home/anilkag/data/imdb-movie-dataset/labeledTrainData.tsv'

### Read the data

In [3]:
def read_data(filename, SEPARATOR = '\t'):
    data_X = []
    data_y = []
    with open(filename) as fp:
        # Skip the header line (id \t sentiment \t review)
        fp.readline() 

        for line in fp:
            tokens = line.split(SEPARATOR)
            data_X.append( tokens[2] )
            data_y.append( int(tokens[1]) )
    return data_X, data_y

data_X, data_y = read_data(data_path)

### Review example

In [4]:
idx=0
print(data_y[idx])
print(data_X[idx])

1
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally s

### Split data into train/valid/test and preprocess

In [5]:
# Split into train/validation/test
print('# of train points = ', len(data_X))
y = np.array(data_y)
X_train, X_test, y_train, y_test = train_test_split(data_X, y, test_size=0.25, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Preprocess using sklearn utilities
vectorizer = TfidfVectorizer()
def preprocess_data(data, train_data=False):
    if train_data:
        X = vectorizer.fit_transform(data)
    else:
        X = vectorizer.transform(data)
    return X
    
X_train = preprocess_data(X_train, train_data=True)
X_valid = preprocess_data(X_valid)
X_test  = preprocess_data(X_test)

data_splits = [(X_train, y_train), (X_valid, y_valid), (X_test, y_test)]
data_splits_name = ['train', 'valid', 'test']
for i in range(len(data_splits)):
    _x, _y = data_splits[i]
    _name  = data_splits_name[i]
    print(_name, ' X.shape = ', _x.shape)
    print(_name, ' y.shape = ', _y.shape)

# of train points =  25000
train  X.shape =  (18750, 66639)
train  y.shape =  (18750,)
valid  X.shape =  (3125, 66639)
valid  y.shape =  (3125,)
test  X.shape =  (3125, 66639)
test  y.shape =  (3125,)


In [6]:
# Train a classifier (note that this should to switch to something more fancier)
#clf = RandomForestClassifier(n_estimators = 100)
clf = LogisticRegression()
clf = clf.fit( X_train, y_train )

In [7]:
# Write utility to compute all the performance metrics
# Evaluate the basic classifier on test data
def compute_performance_metrics(clf, X, y):
    accuracy = clf.score(X, y)
    
    p = pickle.dumps(clf)
    model_size = sys.getsizeof(p)
    
    print('Evaluation..')
    print('Accuracy = ', 100*accuracy)
    print('Model size = ', model_size/1024, ' KB' )
    
compute_performance_metrics(clf, X_test, y_test)

Evaluation..
Accuracy =  88.96
Model size =  521.490234375  KB


In [8]:
# Tune hyper-parameters on the validation data and then evaluate on test data

In [9]:
# Bonus tip : Sparsification of the classifier?

In [ ]:
# Tips : 
# 1) Do not preprocess all the data before splitting into train/valid/test --> your stats should come from the training data
# 2) Use the test data only for evaluation --> NOT EVEN FOR HYPER-PARAMs TUNING
# 3) Start with simple classifiers before moving on the complex function classes --> Keep it simple stupid = KISS
# 4) Normalize the data (sample procedure applied to train/valid/test) :
#    - helps training algorithms and achieves better test performance